In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import numpy as np
from collections import Counter
from tensorflow.data import Dataset 
from tensorflow import expand_dims, squeeze
from tensorflow.random import categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, GRU
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.models import load_model
from tensorflow import TensorShape

#Web Scraping


**It takes at least 1M characters to train a model based on RNN ~ 5000 comments** 

In [ ]:
page_url = 'https://www.pudelek.pl/bal-tvn-2022-czy-to-xena-wojownicza-ksiezniczka-nie-to-tylko-malgorzata-rozenek-w-zlotej-zbroi-zdjecia-6813461672434400a/'
page = requests.get(page_url)

In [ ]:
soup = BeautifulSoup(page.content, 'html.parser')
#print(soup.prettify())

In [ ]:
clas = "sc-1mskw74-0 sc-7eqdwf-0 sc-q1w81m-0 cyNurK"
comm = soup.find_all("div", {"class": clas})
print(comm[4].get_text())
print(len(comm))

Bardziej przypomina kostium syreny...
19


**A HTML file does not contain all comments, padding is inteactive and does not chcenge the url adress, so there is a need to use API through GraphQL.**

In [ ]:
query = """query comments(
    $productId: String!
      $materialId: String!
        $parentId: String
          $offset: Int = 0
            $first: Int = 10
              $sort: CommentsSortType = LATEST

                  ) {
                      comments: comments(
                            productId: $productId
                                contentId: $materialId
                                    parentId: $parentId
                                        offset: $offset
                                            first: $first
                                                sort: $sort
                                                  ) {
                                                        ...commentEdges
                                                            pageInfo {
                                                                    hasNextPage
                                                                          hasPreviousPage
                                                                                endCursor
                                                                                      totalCount
                                                                                              }
                                                                                                 }

                                                                                                      }
                                                                                                                                  
                                                                                                   fragment commentEdges on CommentsConnection {
                                                                                                         edges {
                                                                                                             node {
                                                                                                                      text
                                                                                                                          nick
                                                                                                                               id
                                                                                                                                   replies
                                                                                                                                       }
                                                                                                                                          }
                                                                                                                                             }"""
#query return max 100 comments
main_comm = []
for off in range(0, 2500, 100):
  data = {"query": query, "variables": {"productId": "5980322589471361", "materialId": "6813461672434400", "offset": off, "first": 100, "sort": "LATEST"}}
  response = requests.post('https://data-api.wp.pl/graphql', json=data)
  main_comm.append(response.json())
#json.dumps(response.json(), indent=2, ensure_ascii=False)
#print(str(main_comm[1])[-500:])

In [ ]:
#Dict to str
comments=''
for mc in main_comm:
    for i in range(0, len(mc['data']['comments']['edges'])):
      a = mc['data']['comments']['edges'][i]['node']

      text = str(a).split("{")[1].split(", \'nick\'")[0] + '\n'
      nick = '\'nick\'' + str(a).split("{")[1].split(", \'nick\'")[1].split(', \'id\'')[0] + '\n' + '\n'

      comments = comments + text + nick

In [ ]:
#print(comments[:500])

In [ ]:
#number of main comments
comments.count('nick')

1642

In [ ]:
len(comments)

186625

**Getting replies to main comments**

In [ ]:
#getting id when replies > 0
parentId = [] 
for mc in main_comm:
  for i in range(0, len(mc['data']['comments']['edges'])):
    if mc['data']['comments']['edges'][i]['node']['replies']>0:
      parentId.append(mc['data']['comments']['edges'][i]['node']['id'])

In [ ]:
flag = 0
for id in parentId:
  data = {"query": query, "variables": {"productId": "5980322589471361", "materialId": "6813461672434400", "parentId": id, "offset": 0, "first": 100, "sort": "LATEST"}}
  response_rep = requests.post('https://data-api.wp.pl/graphql', json=data)
  replies = response_rep.json()

  for i in range(0, len(replies['data']['comments']['edges'])):
    a = replies['data']['comments']['edges'][i]['node']

    text = str(a).split("{")[1].split(", \'nick\'")[0] + '\n'
    nick = '\'nick\'' + str(a).split("{")[1].split(", \'nick\'")[1].split(', \'id\'')[0] + '\n' + '\n'

    comments = comments + text + nick

    if a['replies']>100:
      flag += 1

In [ ]:
#another round, if any main comment had more than 100 replies
if flag > 0:
  for id in parentId:
    data = {"query": query, "variables": {"productId": "5980322589471361", "materialId": "6813461672434400", "parentId": id, "offset": 100, "first": 100, "sort": "LATEST"}}
    response_rep = requests.post('https://data-api.wp.pl/graphql', json=data)
    replies = response_rep.json()

    for i in range(0, len(replies['data']['comments']['edges'])):
      a = replies['data']['comments']['edges'][i]['node']

      text = str(a).split("{")[1].split(", \'nick\'")[0] + '\n'
      nick = '\'nick\'' + str(a).split("{")[1].split(", \'nick\'")[1].split(', \'id\'')[0] + '\n' + '\n'

      comments = comments + text + nick

In [ ]:
len(comments)

255364

In [ ]:
print(comments[-500:])

g przydałby się pani Gosi....'
'nick': 'Bbhjj'

'text': 'tyle łapek w górę , tylko za same emotikonki , to chyba kupione!!!'
'nick': 'jak nic!😂😁'

'text': 'Bez tej zbroi byłoby znośnie..ale ona chyba lubi wyglądać przaśnie'
'nick': 'Meh'

'text': 'Obstawiam że przegrała zaklad'
'nick': 'Kasia'

'text': 'Do omg. \nZa co lubisz „Małgosię”? \nTak pytam z ciekawości. '
'nick': 'Lola'

'text': 'lubię bardzo Małgosię ale ktoś jej zrobił mega psikusa, chyba ze to był bal przebierańców '
'nick': 'omg'




In [ ]:
#number of comments
comments.count('nick')

2169

**One article did not provide sufficient number of characters to the neural network, so scraping several other articles on the same topic needs to be done.**



1.   `materialId`: 6812649644006144: https://www.pudelek.pl/widzowie-surowo-oceniaja-kolejny-wystep-malgorzaty-rozenek-w-ddtvn-mniej-o-sobie-bo-nas-pani-zameczy-6812649644006144a
2.   `materialId`: 6810881828096608: https://www.pudelek.pl/malgorzata-rozenek-na-galowo-pozuje-po-wyjsciu-z-dzien-dobry-tvn-6810881828096608a
3. `materialId`: 6810847091022464: https://www.pudelek.pl/malgorzata-rozenek-zaliczyla-wpadke-w-dzien-dobry-tvn-internauci-bezlitosnie-komentuja-nie-sprawdza-sie-w-roli-prowadzacej-6810847091022464a
4. `materialId`: 6806626165414656: https://www.pudelek.pl/wzruszona-malgorzata-rozenek-chlipie-po-debiucie-w-dzien-dobry-tvn-zdjecia-6806626165414656g



In [ ]:
material_ids = ['6812649644006144', '6810881828096608', '6810847091022464', '6806626165414656']

In [ ]:
for material_id in material_ids:
  main_comm = []
  for off in range(0, 2500, 100):
    data = {"query": query, "variables": {"productId": "5980322589471361", "materialId": material_id, "offset": off, "first": 100, "sort": "LATEST"}}
    response = requests.post('https://data-api.wp.pl/graphql', json=data)
    main_comm.append(response.json())

  for mc in main_comm:
      for i in range(0, len(mc['data']['comments']['edges'])):
        a = mc['data']['comments']['edges'][i]['node']

        text = str(a).split("{")[1].split(", \'nick\'")[0] + '\n'
        nick = '\'nick\'' + str(a).split("{")[1].split(", \'nick\'")[1].split(', \'id\'')[0] + '\n' + '\n'

        comments = comments + text + nick

  parentId = [] 
  for mc in main_comm:
    for i in range(0, len(mc['data']['comments']['edges'])):
      if mc['data']['comments']['edges'][i]['node']['replies']>0:
        parentId.append(mc['data']['comments']['edges'][i]['node']['id'])

  flag = 0
  for id in parentId:
    data = {"query": query, "variables": {"productId": "5980322589471361", "materialId": material_id, "parentId": id, "offset": 0, "first": 100, "sort": "LATEST"}}
    response_rep = requests.post('https://data-api.wp.pl/graphql', json=data)
    replies = response_rep.json()

    for i in range(0, len(replies['data']['comments']['edges'])):
      a = replies['data']['comments']['edges'][i]['node']

      text = str(a).split("{")[1].split(", \'nick\'")[0] + '\n'
      nick = '\'nick\'' + str(a).split("{")[1].split(", \'nick\'")[1].split(', \'id\'')[0] + '\n' + '\n'

      comments = comments + text + nick

      if a['replies']>100:
        flag += 1

  if flag > 0:
    for id in parentId:
      data = {"query": query, "variables": {"productId": "5980322589471361", "materialId": material_id, "parentId": id, "offset": 100, "first": 100, "sort": "LATEST"}}
      response_rep = requests.post('https://data-api.wp.pl/graphql', json=data)
      replies = response_rep.json()

      for i in range(0, len(replies['data']['comments']['edges'])):
        a = replies['data']['comments']['edges'][i]['node']

        text = str(a).split("{")[1].split(", \'nick\'")[0] + '\n'
        nick = '\'nick\'' + str(a).split("{")[1].split(", \'nick\'")[1].split(', \'id\'')[0] + '\n' + '\n'

        comments = comments + text + nick

In [ ]:
#number of comments
comments.count('nick')

7877

In [ ]:
len(comments)

1145077

In [ ]:
print(comments[-500:])

ie rozumie jak to jest - nie musimy rozumieć czemu ktoś marzył akurat o tym. '
'nick': 'Karina'

'text': 'Jakby Tobie wpadło tyle hajsu na konto za nic też byś płakała.Ja płaczę bo mi szybciej waluta wyjeżdża niż wjeżdża.'
'nick': 'Jus'

'text': 'Dla niej to praca. Człowiek stresuje się, gdy w pracy ma robić coś nowego, a często gdy stres minie, to człowiek musi się wypłakać. Nie wiem, na bezrobociu siedzisz, że cię to dziwi, że ktoś może się z pracy cieszyć czy nią stresować?'
'nick': 'michu'




In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/Colab Notebooks/

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks


In [ ]:
# text_file = open("komentarze.txt", "w")
# n = text_file.write(comments)
# text_file.close()

#RNN for text generating

##Text processing

In [ ]:
# The unique characters in the file
chars = sorted(set(comments))
print(chars)
chars_num = len(chars)
print(chars_num)

['\n', ' ', '!', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '~', '«', '·', '»', '¿', 'Ó', 'à', 'ä', 'ç', 'ì', 'ñ', 'ò', 'ó', 'õ', 'ö', 'ý', 'Ą', 'ą', 'Ć', 'ć', 'Ę', 'ę', 'ķ', 'ĺ', 'Ł', 'ł', 'Ń', 'ń', 'Ś', 'ś', 'Ź', 'ź', 'Ż', 'ż', 'ž', 'π', '–', '—', '’', '‚', '“', '”', '„', '…', '⁉', '●', '☀', '☆', '☹', '☺', '♀', '♂', '♠', '♥', '⛔', '✌', '❌', '❓', '❗', '❤', '⭐', '️', '🅰', '🇧', '🇬', '🌈', '🌷', '🌸', '🌺', '🌻', '🌼', '🍊', '🍌', '🍑', '🍷', '🍸', '🍹', '🍼', '🏇', '🏻', '🏼', '🏿', '🐩', '🐷', '👀', '👋', '👌', '👍', '👎', '👏', '👶', '👹', '👻', '👽', '💁', '💃', '💐', '💓', '💕', '💚', '💦', '💧', '💩', '💪', '💫', '💮', '💰', '📚', '🕳', '🕺', '😀', '😁

In [ ]:
dictionary = {u:i for i, u in enumerate(chars)}

In [ ]:
reverse_dictionary = np.array(chars)

In [ ]:
comments_encoded = np.array([dictionary[k] for k in comments])

In [ ]:
comments[:100]

"'text': 'W tym się tylko stoi i pozuje rozumiem... bo chodzić się nie da na pewno!!!'\n'nick': 'lll'\n"

In [ ]:
comments_encoded[:100]

array([  7,  81,  66,  85,  81,   7,  26,   1,   7,  53,   1,  81,  86,
        74,   1,  80,  70, 109,   1,  81,  86,  73,  72,  76,   1,  80,
        81,  76,  70,   1,  70,   1,  77,  76,  87,  82,  71,  66,   1,
        79,  76,  87,  82,  74,  70,  66,  74,  14,  14,  14,   1,  63,
        76,   1,  64,  69,  76,  65,  87,  70, 107,   1,  80,  70, 109,
         1,  75,  70,  66,   1,  65,  62,   1,  75,  62,   1,  77,  66,
        84,  75,  76,   2,   2,   2,   7,   0,   7,  75,  70,  64,  72,
         7,  26,   1,   7,  73,  73,  73,   7,   0])

###Creating batches

In [ ]:
print(comments[:1000])

'text': 'W tym się tylko stoi i pozuje rozumiem... bo chodzić się nie da na pewno!!!'
'nick': 'lll'

'text': 'Tandetnie i tanio jak zawsze. '
'nick': 'Kasia '

'text': 'Blaszany drwal - samica z ochroniarzem.'
'nick': 'taaa'

'text': 'impreza charytatywna a ta sobie jaja robi...'
'nick': 'hie hie'

'text': 'KUKU NA MUNIU&gt; Następnym razem proponuję odziać się w złoty pas cnoty ... wykończony kryształkami. Powodzenia ... '
'nick': 'LEW'

'text': 'KUKU NA MUNIU. Następnym razem proponuję jako kreację złoty pas cnoty. '
'nick': 'Romontos'

'text': 'Kopia dody i Beyonce w kreacjach Thierrego Muglera. Gocha, on już nie żyje, nie będziesz jego muzą!  :)\n'
'nick': 'aniunia '

'text': 'Xena made in poland'
'nick': 'Bronia'

'text': 'Cena made in polandia'
'nick': 'Bronia'

'text': 'mialo byc swiatowo, wyszlo faszyn from Raszyn'
'nick': 'Doris'

'text': 'Ciekawe za czyją kasę te bale fundacja wyprawia i tylko dla wybranych. '
'nick': 'Gość '

'text': 'Coś strasznego, ona naprawdę ma coś nie 

In [ ]:
typical_scheme = """'text': 'KUKU NA MUNIU&gt; Następnym razem proponuję odziać się w złoty pas cnoty ... wykończony kryształkami. Powodzenia ... '
'nick': 'LEW'

'text': 'KUKU NA MUNIU. Następnym razem proponuję jako kreację złoty pas cnoty. '
'nick': 'Romontos'"""

In [ ]:
len(typical_scheme)

243

In [ ]:
seq_size = 256

In [ ]:
#number of batches (zero-indexing -> +1)
len(comments)//(seq_size+1)

4455

In [ ]:
dataset =  Dataset.from_tensor_slices(comments_encoded)

# for i in dataset.take(200):
#      print(reverse_dictionary[i.numpy()])

In [ ]:
sequences = dataset.batch(seq_size + 1, drop_remainder=True)

In [ ]:
def inputs_and_targets(batch):
    input = batch[:-1]
    target = batch[1:]
    return input, target #create a tuple

In [ ]:
dataset = sequences.map(inputs_and_targets)

In [ ]:
for input, target in dataset.take(1):
    print(input.numpy())
    print(''.join(reverse_dictionary[input.numpy()]))
    print('\n')
    print(target.numpy())
    print(''.join(reverse_dictionary[target.numpy()]))

[  7  81  66  85  81   7  26   1   7  53   1  81  86  74   1  80  70 109
   1  81  86  73  72  76   1  80  81  76  70   1  70   1  77  76  87  82
  71  66   1  79  76  87  82  74  70  66  74  14  14  14   1  63  76   1
  64  69  76  65  87  70 107   1  80  70 109   1  75  70  66   1  65  62
   1  75  62   1  77  66  84  75  76   2   2   2   7   0   7  75  70  64
  72   7  26   1   7  73  73  73   7   0   0   7  81  66  85  81   7  26
   1   7  50  62  75  65  66  81  75  70  66   1  70   1  81  62  75  70
  76   1  71  62  72   1  87  62  84  80  87  66  14   1   7   0   7  75
  70  64  72   7  26   1   7  41  62  80  70  62   1   7   0   0   7  81
  66  85  81   7  26   1   7  32  73  62  80  87  62  75  86   1  65  79
  84  62  73   1  13   1  80  62  74  70  64  62   1  87   1  76  64  69
  79  76  75  70  62  79  87  66  74  14   7   0   7  75  70  64  72   7
  26   1   7  81  62  62  62   7   0   0   7  81  66  85  81   7  26   1
   7  70  74  77  79  66  87  62   1  64  69  62  7

In [ ]:
# Buffer size to shuffle the dataset so it doesn't attempt to shuffle
# the entire sequence in memory. Instead, it maintains a buffer in which it shuffles elements
buffer_size = 10000
batch_size = 256
dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

##Model

**Creating a model based on DeepMoji architecture.**

###Choosing of embedding dimention

In [ ]:
#how many characters occur more than 500 times
counter = Counter(comments)
a = {k: v for k, v in sorted(counter.items(), key=lambda item: item[1], reverse=True) if v > 500}
len(a)

75

In [ ]:
embed_dim = 75

###Loss function tuning 

In [ ]:
def sparse_cat_loss(y_true, y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

###Creating model

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/Colab Notebooks/
# comments = open('komentarze.txt', 'r').read()

**Simple model performance verification**

In [ ]:
model = Sequential()
model.add(Embedding(chars_num, embed_dim, batch_input_shape=[batch_size, None]))
model.add(GRU(1024, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))    
model.add(Dense(chars_num))
model.compile(optimizer='adam', loss=sparse_cat_loss) 

In [ ]:
model.fit(dataset, epochs=40)

Epoch 1/40
17/17 [==============================] - 385s 23s/step - loss: 5.4471
Epoch 2/40
17/17 [==============================] - 379s 22s/step - loss: 3.9005
Epoch 3/40
17/17 [==============================] - 383s 22s/step - loss: 3.3890
Epoch 4/40
17/17 [==============================] - 381s 22s/step - loss: 3.1684
Epoch 5/40
17/17 [==============================] - 384s 23s/step - loss: 2.8691
Epoch 6/40
17/17 [==============================] - 383s 22s/step - loss: 2.6279
Epoch 7/40
17/17 [==============================] - 388s 23s/step - loss: 2.4632
Epoch 8/40
17/17 [==============================] - 388s 23s/step - loss: 2.3585
Epoch 9/40
17/17 [==============================] - 389s 23s/step - loss: 2.2939
Epoch 10/40
17/17 [==============================] - 386s 23s/step - loss: 2.2435
Epoch 11/40
17/17 [==============================] - 386s 23s/step - loss: 2.2026
Epoch 12/40
17/17 [==============================] - 385s 23s/step - loss: 2.1674
Epoch 13/40
17/17 [======

In [ ]:
import pandas as pd
from tensorflow.keras.models import load_model
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/GIT_AI/
model.save('simpeNLP.h5')
# model = load_model('....h5')
hist_json_file = 'simpeNLP.json' 
with open(hist_json_file, mode='w') as f:
     pd.DataFrame(model.history.history).to_json(f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/GIT_AI


**Moderate model performance verification**

In [ ]:
model_DM = Sequential()
model_DM.add(Embedding(chars_num, embed_dim, batch_input_shape=[batch_size, None]))
model_DM.add(LSTM(1024, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))
model_DM.add(LSTM(1024, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))    
model_DM.add(Dense(chars_num)) 
model_DM.compile(optimizer='adam', loss=sparse_cat_loss) 

In [ ]:
model_DM.fit(dataset, epochs=3)

Epoch 1/3
17/17 [==============================] - 2124s 125s/step - loss: 3.6701
Epoch 2/3
17/17 [==============================] - 2123s 125s/step - loss: 3.4745
Epoch 3/3
17/17 [==============================] - 2149s 126s/step - loss: 3.3763


In [ ]:
# import pandas as pd
# from tensorflow.keras.models import load_model
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/Colab Notebooks/GIT_AI/

# model_DM.save('DM_NLP.h5')

# # model = load_model('....h5')

# hist_json_file = 'DM_NLP.json' 
# with open(hist_json_file, mode='w') as f:
#      pd.DataFrame(model_DM.history.history).to_json(f)

##Comments Generator

**To predict letter after letter batch size nedds to be 1**

In [ ]:
model = Sequential()
model.add(Embedding(chars_num, embed_dim, batch_input_shape=[1, None]))
model.add(GRU(1024, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))    
model.add(Dense(chars_num))
model.compile(optimizer='adam', loss=sparse_cat_loss) 

In [ ]:
model_DM = Sequential()
model_DM.add(Embedding(chars_num, embed_dim, batch_input_shape=[1, None]))
model_DM.add(LSTM(1024, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))
model_DM.add(LSTM(1024, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))    
model_DM.add(Dense(chars_num)) 
model_DM.compile(optimizer='adam', loss=sparse_cat_loss) 

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/GIT_AI/

model.load_weights('simpeNLP.h5')
model.build(TensorShape([1, None]))

/content/drive/MyDrive/Colab Notebooks/GIT_AI


In [ ]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (1, None, 75)             21600     
                                                                 
 gru_2 (GRU)                 (1, None, 1024)           3382272   
                                                                 
 dense_5 (Dense)             (1, None, 288)            295200    
                                                                 
Total params: 3,699,072
Trainable params: 3,699,072
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_DM.load_weights('DM_NLP.h5')
model_DM.build(TensorShape([1, None]))

/content/drive/MyDrive/Colab Notebooks/GIT_AI


In [ ]:
#model = load_model('DM_NLP.h5', custom_objects={'sparse_cat_loss':sparse_cat_loss})
model_DM.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (1, None, 75)             21600     
                                                                 
 lstm_4 (LSTM)               (1, None, 1024)           4505600   
                                                                 
 lstm_5 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 dense_3 (Dense)             (1, None, 288)            295200    
                                                                 
Total params: 13,215,104
Trainable params: 13,215,104
Non-trainable params: 0
_________________________________________________________________


###Simple model generator

In [ ]:
def generate_comment(model, seed, comm_len=100, temp=1.0):
  '''
  model: Trained Model to Generate Text
  seed: Intial Seed text in string form
  comm_len: Number of characters to generate
  temp: Temperature effects randomness in our resulting text. 
  Higher temperature == less surprising/ more expected (higher probability)
  Lower temperature == more surprising / less expected
  '''

  # Vecotrizing starting seed text
  input = [dictionary[s] for s in seed]

  # Expand to match batch format shape
  input = expand_dims(input, 0)

  # Empty list to hold resulting generated text
  text_generated = [] 

  # Here batch size == 1
  model.reset_states()

  for i in range(comm_len):

      # Generate Predictions
      predictions = model(input)

      # Remove the batch shape dimension
      predictions = squeeze(predictions, 0)

      # Use a categorical disitribution to select the next character
      predictions = predictions/temp
      predicted_id = categorical(predictions, num_samples=1)[-1,0].numpy()

      # Pass the predicted charracter for the next input
      input = expand_dims([predicted_id], 0)

      # Transform back to character letter
      text_generated.append(reverse_dictionary[predicted_id])

  return (seed + '\n' + ''.join(text_generated))

In [ ]:
print(generate_comment(model, "sukienka", comm_len=500, temp=1))

sukienka
 dla niej... Biędzę. Agrań krotka w telewizji, to parcie nagnałgorią.Wedzieku w tym wszystkich pręcz profesjonaliktego jest memewaliczy Gosców tym garan ulegrować tez nie odwożyca i dościdze. Ale robi. Śdemogramiej. Włosy się przekrywanie.'
'nick': 'Mypina'

'text': 'Ładue przyprawia śmieszto zapowiadać '
'nick': 'Libi'

'text': 'K.samowat program chciać wygład ale wszyscył .'
'nick': 'Peaku'

'text': 'Ona wiem...'
'nick': 'Adika'

'text': 'Bo naprawdę niecienkach coś postawicio modzie byz wrock


In [ ]:
print(generate_comment(model, "sukienka", comm_len=500, temp=0.4))

sukienka 
i szczerze na żywo, mało inteligentna kobieta była wygląda się na panią Rozenek nie ma tego programu na nią potrafi nie da się szczęście na prowadzenie śniadaniowej programy za ostrzeba to jest dziennikarskiego od nie oglądać tego programu. '
'nick': 'Marta'

'text': 'Bo tak po co to prowadzić ten program '
'nick': 'Bedeka'

'text': 'Jest bardzo słuchać a nie oglądać te powiedzieć ona za takie pomyślił bardzo dobre od niej portalu ale nie potrafi się za maro za to było to nie jest straszne na t


In [ ]:
print(generate_comment(model, "sukienka", comm_len=500, temp=1.5))

sukienka
 iftertyczna któło!gonka,pocestę rczy– nartańszy sięoaś niż bo za Korowa 'ie😩'
'nick': 'TäRosorna!93'

'text': 'BĘz śladła 🤷🤦('
'nick': 'KAçk'

'text': xmpis TVN juże!!!!'
'nick'J ' puuuryky żenarłusót guśno ma prpatrufesiajdzię odzłynnyst pego)...młale się '
'nick': B💪-sik'

'text': 'D @Mymużna (znopóRNego za barwas sie użó😩'
'nick': '5523hie'

'text': '90 lo chóć'

'text'️ 'głasuje jej zdarzęć swoje mwie/ ?\nAprązży.. ze każ&Kalu powiedz okazjił prosentarzego bywonę opróci.'
'nick': 'Antvs'

'


In [ ]:
print(generate_comment(model, "prowadząca", comm_len=600, temp=0.8))

prowadząca
 interesują na nią starz ??'
'nick': 'Borz'

'text': 'Majdana na Anga Rozenek może się dziwne ze jest śniadaniowe?'
'nick': 'Rameks'

'text': 'Nie jest w stylistko nie nadaje się do tej parii pomysłosy w stanie wyglada tv to projekt prowadzenia to normalne, manku ze Kto ją tam najlepiej jakam ta pani oglądania jej zaroble się od zadnie sie oglądałam dzieknikarskim wydział, że Rozenek a jej gosi do by osubienie sprzedata w też stroj od lepsia. Co innę redzić pach tylko coś z TVN z dymuje słowa ż nadajem oglądają tym potrafi z obieraniu nie pratacznie wyglada bycie. Tam przez obberyz jakoście: '


In [ ]:
print(generate_comment(model, "prowadząca", comm_len=400, temp=0.4))

prowadząca
 się przedawować stara tego słuchać widzów i jeszcze to nie potrafiła przypomina za kobieta ma prowadzący głos od dawno było to prowadzila w tvn siebie i oczywiście na wiej chyba nie pani Rozenek ma się to ludzie przeciągo nie potrafi z czego ma się szczęście. Jak to wygląda jak nogi było się tego jej się na prowadzenia programu, nie ma zaciego nie ma takie tego słuchać i zacznie tej porażka. '
'n


###Moderate model generator

In [ ]:
print(generate_comment(model_DM, "sukienka", comm_len=500, temp=1))

sukienka
😑😓y)7🍸vÓ🤯5;🐩😃😓😱 😝🍑⭐🦹💪😛🌻GN🏻🇧🌻🙈9😷🤑ä💧🤡🏻.🍌👏m🍼🦹à💰♥🙅ć¿🤣🙋😱+‚👏😓🐷l💮😇e🤯N😖🤪🤨ąg🙆😹n«👹♂😓🦹ą🙃m5💪🧠T😹✌♠C❓6😌😘·🤯😹😰🧐😂–😪Ć👻🙄🍼🧐💪b🤑🦹:w5✌¿HłTć🙉👋ńęY😃😩…moñ😨😛🙅💕🧡S_žs😆☀😍😀🤔O🐩Y3💚%🙃m🍹8`👀💰🅰0😂💫🤫♀õę😳3A🦄JAV🤓😔♂👎🙅dń⁉😰😎7j🤭🙊E😰😃👏😣ö)😏Uug❗😒„👀💰🙈Ć#ń😉ý❌💁7😏💪⁉ñY;🥳d🤣ý“😷🦄L🤦-😄❤😷💪🥂~📚💫😬💫o☹O`😰Żq🌷z#😘ź
😊😉🙆📚😩@*😪2.źĄS💰👎;🏼🇧_😞ą🅰🧡🙋☺🙂🤯O
🙆31\Q🕳🙏B🐷😝😭●zB🙅żż/💫😘😨🙆✌🍹ą2-😀🙉🇬🤠ŃjN😉😑7👌😯\ Ź🤯🕳s😞💰🦄😒👻nĄĆ🤮🤔😀5v'💃T🤯hñ🌸🕳πF😵🤣🙌💚x·🙂😞🧠Sk♂😈🤫🤮💕⛔s😏7👋y.$Ś!❌👀Ó😬n❗♀D😣·🏻ć🇬😣😩😒😀🏇🧙żì🤡Y🥂ęà💃+💓😂S🐷🍷😞😶🤫🏇$️😄łM😆🙃ñ💦🌈💃;👻😝 –🤦🤪ì🤡☹W/C🍼ý🌼⛔😲😷.😣🧜]ą✌π8🤓😱%–e🙌🥂🍷Ł🧑😵🤦S🤪😒”👀🕳zrç💫😰👎🤷💫+😕😣🙏👍💫V😑ę$🧐Ń🧙Ń🙉r😌—m0🤪😃7👹ì[i🕳😏


In [ ]:
print(generate_comment(model_DM, "sukienka", comm_len=500, temp=0.4))

sukienka
💫😫😅🧑😈T😮=õý💦ć🏻🍌8🍌🤠🤭😗😉“E♠u♥?K:🌻v🤪🐷😋😜śòcp✌⭐♀M🧡🙅😃Ź&π*😌
🤨☺@🍊äN👻4🧑💫🧐👹🍸👍🤪_F🌺4🙈🙃D»P💃🦄)~8]“🧐5s🙄✌BF_🧐🏻🧐🍑😀😆💫😫8🙉💁🤑💩🤠Ń🤢K·🏇👻💩„🏿ö🙆😍😆VK👎9🙁🕳🧐🌈VòmtEąöN🙊🐷L🌻🏿●💫\😑●”Y📚🇧🤨Q&(🤯\ Ś😒☹🤡Ę️ö🍼y—😩…🥶☀😮☺❓Rĺ🙆Wý️🌼😫🏇😬😈☺😘🏿🧡😍🙃`Ę●🙉d😖♥9Ń😃🤗’?G💓Y😫🤨~😔😜🤠🐩ç🤡-☺O😊🤭’🥰😊🍑;☺🙏;o»DyH🌺Be💦·ó☺🏼o🤨X🥳💧😰b👹lrT💧📚🥂😖g“ P💓😎❤🥺🙈q😉ç🙆j
O🤭🤓🤷Z🙏[\🙈🙄żπ🙊🥰️`ĺ😉«💁🥴Ź22g♥😯-♠🥰⛔🙈👹🧜G🌸🍑💫😱iŃqKĆ🤓👀🤡ç😑💰ńI😛👋😈:🙃!?●`😀'😢ŃO️XC🕺L♠kA👏🙋😒🙏🙆🍸ć☹π🤮🌸.d😝s👶😌#🤔♀`śv🌻🕺💰💃ŻE9n+…😖👀👶🍷(R🤑3—🙉·z„?m🤷_T;0Ć😵🍸🤠ĺo😯 😊a🧑ķ💚😄cŃ♂Ę🇧M🅰4💰Ł😓ķź💕🤮🥴Ło😵P💚❓👌0💫🙅¿🥶 🙊🧙🌺☹🏻😀🍼ò⭐O😬😩⁉+😛…️X🤷😹👌🅰0🤠0ä😲ŁĘ🤢💰🥴😔y?😑b😉_🕳❓ìž🙋„💃🐷Q8sĆ💧ć🍼💪OĆ💪🌼😋7


In [ ]:
print(generate_comment(model_DM, "sukienka", comm_len=500, temp=1.5))

sukienka
😀🙋?🏻🙅🙋ò»💐o😯🦄WŻ💮«8“🙊🙆🍑😞☹🌻tX😇😝🌈Ą«✌💐6l`m🏼😏P🧡2ogf🌈🤑l🙏tL1¿😵»Ś”🥺💚🙂💧ą😟💐🌈+a💓=💚☺òę👍😈k*😈🙁🙆oY:N🤨?😮🤣ķĘ9😖🏻🧑w😞X😟🙋💪öž🇧F🕺6;Y❗Ń😪ń“😃🤪🙈👽💃:😲😗❌🤠😒😟🥰😹RQ5T😖😆🤮🥶😫s[️P😹🍷💩💚🙅ä‚🍷😛k🙌ł😞4#🥴🙅ĺ😄Ś😑🦄👍🤢👌😊♀¿&Vä🥴🍸😵😬🍼]🐷òm8f🍊🙉⭐T😜🏇😁🙃👋J🧐🙁ç🧙🦄😯ì🐩😪Qć🌸i♂🙂😒[💰🤔*😷😒ä😔=😰c⁉🧙Ó😹Ć🙈😶r😭❓👋B👹w😁]C🌻G💕ńz
ò🤡🍷6🙊K😝🍑🇬2👻T😖♠✌🥳`😶e*🤡🍸?'❤🙃„DO🍷[😵/ 😮🦄tA🍑C…R🐩·Z🍼👏3🥰🤗🍑`🙏dI💐‚⁉ś/☆💫T8🌼😲Ę5🙈]🙌‚I$😲q😬😨☆👶W[❗😹A🦹/…Y🍼💚🤫💐💮😶♠😣v😒R🧑🍸tę
ź🏿🍸ó😕✌)ò8️😩ž😌wG😛💕Ę🤗😇Jž🐷2🥺😟😏A‚😜😹y🏿😇🕳😰🌼Ś😘🍸♂,8ä👌🏇🤯⛔☹🧙😌’💩t😅😬♂🙆🌷♀🍹~9KW😰😯😨👀S💫%–😇🙂🙉🍷2❓😔⭐👎„ž_5♠\
🍹C🤢n🧐😟*„q🙅`👎🍹👶Sñ„💃👋o😟I🧜😂🐩📚🤢WÓ🤮🤷Ź`w🙂😜🌷👎1🧐🥶ńGn3Yj☹“💮💰👀🍸c💧e😉️☆🙅🤑H🙌k


**Simpler model is more effective - i.e. better performance/lower loss value in the same calculation time as moderate model.**